In [8]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

In [12]:
# Initialising the remaining constants
# units: x [mm],
#        y [mm],
#        phi [°],
#        v_x [mm/s]
#        v_y [mm/s]
#        phi_point [°/s]
#        Ts [s]

Ts = 0.1
A = np.array([[1, 0, 0, Ts, 0, 0],
              [0, 1, 0, 0, Ts, 0],
              [0, 0, 1, 0, 0, Ts],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

stripe_width = 50
q_x = 0.04
q_y = 0.04
q_phi = 1.
q_vx = 75.72
q_vy = 75.72
q_vphi = 200.

#peut-être ajuster Q à chaque iteration en fonction de phi 
Q = np.array([[q_x, 0, 0, 0, 0, 0],
              [0, q_y, 0, 0, 0, 0],
              [0, 0, q_phi, 0, 0, 0],
              [0, 0, 0, q_vx, 0, 0],
              [0, 0, 0, 0, q_vy, 0],
              [0, 0, 0, 0, 0, q_vphi]])

r_x = 0.25
r_y = 0.25
r_phi = 5
r_vx = 75.72
r_vy = 75.72
r_vphi = 200.

speed_conv_factor = 0.3375; #valeur moteur -> mm/s
angular_speed_conv_factor = 0.1923 #delta valeur moteur -> °/s

def kalman_filter(speed_left, speed_right, camera_x, camera_y, camera_phi, camera_flag, x_est_prev, P_est_prev,
                  HT=None, HNT=None, RT=None, RNT=None):
    """
    Estimates the current state using input sensor data and the previous state
    
    param speed_left: measured speed of motor left(Thymio units)
    param speed_right: measured speed of motor right(Thymio units)
    param camera_x: measured x coordinate with the camera
    param camera_y: measured y coordinate with the camera
    param camera_phi: measured phi angle with the camera
    param camera_flag: flag to know if the camera is operational 
    param x_est_prev: previous state a posteriori estimation
    param P_est_prev: previous state a posteriori covariance
    
    return x_est: new a posteriori state estimation
    return P_est: new a posteriori state covariance
    """
    
    ## Prediciton through the a priori estimate
    # estimated mean of the state
    x_est_a_priori = np.dot(A, x_est_prev);
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    #speed 
    speed = (speed_left + speed_right)/2
    speed_angular = (speed_right - speed_left)/2 #si positif -> sens horaire 
    
    if (camera_flag): 
        
        y = np.array([[camera_x],
                      [camera_y],
                      [camera_phi],
                      [speed*speed_conv_factor*cos(x(3))],
                      [speed *speed_conv_factor*sin(x(3))],
                      [speed_angular*angular_speed_conv_factor]])
        H = np.array([[1, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]])
        R = np.array([[r_x, 0, 0, 0, 0, 0],
                      [0, r_y, 0, 0, 0, 0],
                      [0, 0, r_phi, 0, 0, 0],
                      [0, 0, 0, r_vx, 0, 0],
                      [0, 0, 0, 0, r_vy, 0],
                      [0, 0, 0, 0, 0, r_vphi]])
    else:
        # no transition, use only the speed
        y = np.array([[speed*speed_conv_factor*cos(x(3))],
                      [speed *speed_conv_factor*sin(x(3))],
                      [speed_angular*angular_speed_conv_factor]])
        H = np.array([[0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]])
        R = np.array([[r_vx, 0, 0],
                      [0, r_vy, 0],
                      [0, 0, r_vphi]])

    # innovation / measurement residual
    i = y - np.dot(H, x_est_a_priori);
    # measurement prediction covariance
    S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
             
    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
    
    
    # a posteriori estimate
    x_est = x_est_a_priori + np.dot(K,i);
    P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));
     
    return x_est, P_est

In [14]:
x_est = np.array([[0],
                  [0],
                  [0],
                  [0],
                  [100],
                  [0]])
P_est = [1000 * np.ones(6)]
k0 = 55

l_speed = [x["left_speed"] for x in thymio_data]
r_speed = [x["right_speed"] for x in thymio_data]



speed_left = [l_speed[k0 - 1]]
speed_right = [r_speed[k0 - 1]]

for k in tqdm(range(k0, len(thymio_data))):
    speed_left.append(l_speed[k])
    speed_right.append(r_speed[k])
    new_x_est, new_P_est = kalman_filter(speed_left[-1], speed_right[-1], 0, 0, 0, 0, x_est[-1], P_est[-1])
    
    x_est.append(new_x_est)
    P_est.append(new_P_est)

node.send_set_variables(motors(100, 100))
await client.sleep(1)


NameError: name 'thymio_data' is not defined